In [1]:
from datetime import date, datetime, time
from typing import Optional, List
from decimal import Decimal
from omop_alchemy import oa_config
import sqlalchemy as sa
import sqlalchemy.orm as so

# this notebook contains a demonstration for how to dynamically create the 
# repetitive columns and relationships required to support the multiple 
# links between the Concept table and all other CDM tables

# no solution yet on how to abstract away the link further via hybrid property
# if we want to add the mapping attributes dynamically, but the overhead between 
# selecting person.gender_concept.concept_name vs. person.gender_label seems 
# minimal compared to the large amounts of code that can be simplified away
# from sqlalchemy.ext.hybrid import hybrid_property


class Concept_Links():
    # class property of form {label: optional, label2: optional}
    # that will be expanded into the mapped column label_concept_id as 
    # foreign key to concept table and the associated label_concept relationship
    labels = {}

    @classmethod
    def add_concepts(cls):
        for label, opt in cls.labels.items():
            so.add_mapped_attribute(cls, f'{label}_concept_id', so.mapped_column(sa.Integer, sa.ForeignKey('concept.concept_id'), nullable=opt, default=0))
            so.add_mapped_attribute(cls, f'{label}_concept', so.relationship("Concept", primaryjoin=f"{cls.__tablename__}.c.{label}_concept_id==Concept.concept_id"))

In [2]:


from omop_alchemy.db import Base

class Person_Dynamic(Base, Concept_Links):
    __tablename__ = 'person_dynamic'

    # now we can just provide a list of linked concepts and whether or not they are required
    # and the mapped columns will be created - main reason to do this is if we decide to change
    # the way we map these down the line, it only needs to be done in one place
    labels = {'gender': False, 'ethnicity': False, 'race': False, 'gender_source': False, 'ethnicity_source': False, 'race_source': False}

    person_id: so.Mapped[int] = so.mapped_column(primary_key=True, autoincrement=True)
    year_of_birth: so.Mapped[Optional[int]] = so.mapped_column(sa.Integer)
    month_of_birth: so.Mapped[Optional[int]] = so.mapped_column(sa.Integer)
    day_of_birth: so.Mapped[Optional[int]] = so.mapped_column(sa.Integer)
    birth_datetime: so.Mapped[Optional[datetime]] = so.mapped_column(sa.DateTime)
    death_datetime: so.Mapped[Optional[datetime]] = so.mapped_column(sa.DateTime)

    def __repr__(self):
        return f'Person: person_id = {self.person_id}'
    
    def get_approximate_dob(self):
        if self.year_of_birth is None:
            return None
        day = self.day_of_birth or 1
        month = self.month_of_birth or 1
        return datetime(self.year_of_birth, month, day)

    def age_calc(self, age_at, selected_dob):
        if selected_dob is None:
            return {}
        age = (age_at - selected_dob).days
        years = age // 365
        days = age % 365
        return {'age_total': age, 'age_years': years, 'age_days': days}

    @property
    def age(self, age_at=None):
        
        if age_at is None:
            age_at = datetime.now()

        if self.death_datetime is not None:
            age_at = min(age_at, self.death_datetime)

        selected_dob = self.birth_datetime or self.get_approximate_dob()

        return self.age_calc(age_at, selected_dob)

In [3]:
# trade-off is having to make sure we call the class function in the right spot

Person_Dynamic.add_concepts()

In [5]:
from omop_alchemy.db import Base
from omop_alchemy.helpers.create_db import create_db 


create_db(Base, oa_config.engine)

In [7]:
# some not so great demo person objects just for the sake of demonstration of dynamic 
# column generation as well as properties and their use in abstracting away calculations
# with multiple columns or column transformations, e.g. age, which can come from one of a 
# few different fields, and should stop counting at time of death

def populate_clinical_demo_data():
    with so.Session(oa_config.engine) as sess:

        gender_concepts = [(8532, 'F', 'FEMALE'), (8507, 'M', 'MALE')]
        ages = [{'year_of_birth': 1950}, 
                {'year_of_birth': 1970, 'day_of_birth': 12, 'month_of_birth': 3},
                {'birth_datetime': datetime(1990, 4, 2)}, 
                {'birth_datetime': datetime.now()},
                {'birth_datetime': datetime(2000, 4, 2), 'death_datetime': datetime(2008, 4, 2)}]

        for n in range(15, 20):
            p = Person_Dynamic(person_id=n, 
                               gender_concept_id=gender_concepts[n%2][0],
                               ethnicity_concept_id=0,
                               race_concept_id=0,
                               **ages[n%5])
            sess.add(p)
        sess.commit()

In [11]:
populate_clinical_demo_data()

In [12]:
with so.Session(oa_config.engine) as sess:
    people = sess.query(Person_Dynamic)

In [13]:
[p.age for p in people]

[{'age_total': 27068, 'age_years': 74, 'age_days': 58},
 {'age_total': 19693, 'age_years': 53, 'age_days': 348},
 {'age_total': 12367, 'age_years': 33, 'age_days': 322},
 {'age_total': 0, 'age_years': 0, 'age_days': 0},
 {'age_total': 2922, 'age_years': 8, 'age_days': 2}]

In [14]:
# this will only work if you have also loaded the related concepts required to reference
[(p.gender_concept_id, p.gender_concept) for p in people]

[(8507, <Concept 8507 - M (MALE)>),
 (8532, <Concept 8532 - F (FEMALE)>),
 (8507, <Concept 8507 - M (MALE)>),
 (8532, <Concept 8532 - F (FEMALE)>),
 (8507, <Concept 8507 - M (MALE)>)]